In [1]:
import numpy as np
from math import *
import random

In [2]:
file = "amazon_cells_labelled.txt"

def data_cutter_1(file: str):
    # code to cut data
    training_data = []
    classes = {}
    vocabulary = {}

    data = open(file, "r")

    c = 0
    w = 0

    for line in data:
        t_line = line \
            .rstrip('\n') \
            .split('\t')

        label = t_line[1]
        sentence = t_line[0]

        # split words in each sentence. future: tokenize and remove filler words
        words = sentence.split()

        doc = (label, words)

        training_data.append(doc)

        for word in words:
            if word not in vocabulary:
                # initialising word in vocabulary
                vocabulary[word] = {"frequency": {label: 1}, "class": [label]}
            else:
                if label in vocabulary[word]["class"]:
                    vocabulary[word]["frequency"][label] += 1

                else:
                    vocabulary[word]["class"].append(label)
                    vocabulary[word]["frequency"][label] = 1

            if label not in classes:
                classes[label] = 1
            else:
                classes[label] += 1
            
            w += 1

        c += 1
        if c % 100 == 0:
            print("on line {}...".format(c))

        training_data.append(doc)

    print('finished on line {} with {} words'.format(c, w))

    return training_data, classes, vocabulary

In [21]:
amazon = "amazon_cells_labelled.txt"
imdb = "imdb_labelled.txt"
yelp = "yelp_labelled.txt"

def data_cutter(*file: str):
    """

    :type file: str
    """
    # code to cut data
    total_data = []
    training_data = []
    test_data = []
    classes = {}
    vocabulary = {}
    word_count = 0

    for f in file:
        data = open(f, "r")

        c = 0
        w = 0

        for line in data:
            t_line = line \
                .rstrip('\n') \
                .split('\t')

            label = int(t_line[1])
            sentence = t_line[0]

            # split words in each sentence. future: tokenize and remove filler words
            words = sentence.split()

            doc = (label, words)

            total_data.append(doc)

            w += len(words)
            
            c += 1

        random.shuffle(total_data)

        training_data = total_data[0:int(0.8 * len(total_data))]
        test_data = total_data[int(0.8 * len(total_data)):]

        print('{} finished on line {} with {} words'
              '\n==========================\n'.format(f, c, w))
        data.close()

        # exit file loop

    for doc in training_data:

        label = doc[0]
        words = doc[1]

        for word in words:
            if word not in vocabulary:
                # initialising word in vocabulary
                vocabulary[word] = {"frequency": {label: 1}, "class": [label]}
            else:
                if label in vocabulary[word]["class"]:
                    vocabulary[word]["frequency"][label] += 1

                else:
                    vocabulary[word]["class"].append(label)
                    vocabulary[word]["frequency"][label] = 1
                    
            for cls in classes:
                if cls not in vocabulary[word]["class"]:
                    vocabulary[word]["frequency"][cls] = 0
                    
                    
                    
            if label not in classes:
                classes[label] = 1
            else:
                classes[label] += 1

            word_count += 1
    # 
    #     c += 1
    #
    # print('{} finished on line {} with {} words'
    #       '\n==========================\n'.format(f, c, w))
    
    print(
        "=====Summary=====\n"
        "Unique words:\t -> {}\n"
        "Total words:\t -> {}\n"
        "Total data:\t -> {}\n"
        "Train data:\t -> {}\n"
        "Test data:\t -> {}\n"
            .format(len(vocabulary),
                    word_count,
                    len(total_data),
                    len(training_data),
                    len(test_data)))

    return total_data, training_data, test_data, classes, vocabulary

In [22]:
tot, train, test, cls, voc = data_cutter(amazon, imdb, yelp)

good = [word for word in voc if '1' in voc[word]["class"]]
bad = [word for word in voc if '0' in voc[word]["class"]]

unq_good = [word for word in good if word not in bad]
unq_bad = [word for word in bad if word not in good]

both = [word for word in voc if word in good and word in bad]

# for word in both:
#     print("{}\t-> {}".format(word, voc[word]))

print(cls)
   

amazon_cells_labelled.txt finished on line 1000 with 10246 words

imdb_labelled.txt finished on line 1000 with 14354 words

yelp_labelled.txt finished on line 1000 with 10894 words

=====Summary=====
Unique words:	 -> 6874
Total words:	 -> 28531
Total data:	 -> 3000
Train data:	 -> 2400
Test data:	 -> 600

{0: 14504, 1: 14027}


In [23]:
cls

{0: 14504, 1: 14027}

In [24]:
for v in voc:
    print("{}\t -> {}".format(v, voc[v]))

The	 -> {'frequency': {0: 194, 1: 185}, 'class': [0, 1]}
bipolarity	 -> {'frequency': {0: 1}, 'class': [0]}
of	 -> {'frequency': {0: 257, 1: 244}, 'class': [0, 1]}
the	 -> {'frequency': {0: 595, 1: 619}, 'class': [0, 1]}
ruthless	 -> {'frequency': {0: 2}, 'class': [0]}
thug	 -> {'frequency': {0: 1}, 'class': [0]}
(one	 -> {'frequency': {0: 1}, 'class': [0]}
minute	 -> {'frequency': {0: 3, 1: 1}, 'class': [0, 1]}
a	 -> {'frequency': {0: 324, 1: 359}, 'class': [0, 1]}
killer,	 -> {'frequency': {0: 2, 1: 0}, 'class': [0]}
next	 -> {'frequency': {0: 8, 1: 4}, 'class': [0, 1]}
Luv's	 -> {'frequency': {0: 1}, 'class': [0]}
diaper	 -> {'frequency': {0: 1}, 'class': [0]}
commercial)	 -> {'frequency': {0: 1}, 'class': [0]}
is	 -> {'frequency': {0: 261, 1: 327}, 'class': [0, 1]}
completely	 -> {'frequency': {0: 9, 1: 3}, 'class': [0, 1]}
unconvincing.	 -> {'frequency': {0: 1}, 'class': [0]}
It	 -> {'frequency': {1: 60, 0: 54}, 'class': [1, 0]}
fits	 -> {'frequency': {1: 6, 0: 1}, 'class': [1, 0]

loud).	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
Extremely	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
Tasty!	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
Seriously,	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
your,	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
kid's	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
highest,	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
superlative	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
form	 -> {'frequency': {1: 3, 0: 1}, 'class': [1, 0]}
cinema	 -> {'frequency': {1: 2, 0: 0}, 'class': [1]}
imaginable.	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
cheap	 -> {'frequency': {0: 10, 1: 0}, 'class': [0]}
olives.	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
satisfied	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
purchase.	 -> {'frequency': {1: 4, 0: 1}, 'class': [1, 0]}
THey	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
cool!	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
peachy-keen.	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
predictable	 ->

special.	 -> {'frequency': {1: 1, 0: 1}, 'class': [1, 0]}
SHOULD	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
hilarious,	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
Eve	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
biggest	 -> {'frequency': {0: 1, 1: 1}, 'class': [0, 1]}
fail	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
us.	 -> {'frequency': {0: 3, 1: 1}, 'class': [0, 1]}
You'd	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
IQ	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
mollusk	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
coming.	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
Samsung	 -> {'frequency': {1: 3, 0: 1}, 'class': [1, 0]}
flipphones.	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
prices.	 -> {'frequency': {1: 2, 0: 0}, 'class': [1]}
dripping	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
grease,	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
edible.	 -> {'frequency': {0: 3, 1: 0}, 'class': [0]}
expected.	 -> {'frequency': {1: 2, 0: 0}, 'class': [1]}
job.	 -> {'frequency': {

questioning	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
glance.	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
Thai.	 -> {'frequency': {1: 1, 0: 0}, 'class': [1]}
Overpriced	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
getting.	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
keypads	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
reach	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
buttons.	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
Bisque,	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
Bussell	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
Sprouts,	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
Risotto,	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
Filet	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
pepper..and	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
tables.	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}
Earbud	 -> {'frequency': {0: 1, 1: 0}, 'class': [0]}


In [7]:
# for word in voc:
#     print('{} -> {}'.format(word, voc[word]['probability']))

KeyError: 'probability'

In [25]:
def probs():

    for word in voc:
        voc[word]["probability"] = {}
        for cl, wfr in voc[word]["frequency"].items():
            cf = cls[cl]
            # wfr = voc[word]["frequency"][cl]
            p = log((wfr + 1)/(cf + len(voc)))

            voc[word]["probability"][cl] = p

    return voc

probs()

{'The': {'class': [0, 1],
  'frequency': {0: 194, 1: 185},
  'probability': {0: -4.6971180762701445, 1: -4.721805609985028}},
 'bipolarity': {'class': [0],
  'frequency': {0: 1},
  'probability': {0: -9.276970454273945}},
 'of': {'class': [0, 1],
  'frequency': {0: 257, 1: 244},
  'probability': {0: -4.417158049912273, 1: -4.446294073153502}},
 'the': {'class': [0, 1],
  'frequency': {0: 595, 1: 619},
  'probability': {0: -3.5798769677685414, 1: -3.5178328056590913}},
 'ruthless': {'class': [0],
  'frequency': {0: 2},
  'probability': {0: -8.871505346165781}},
 'thug': {'class': [0],
  'frequency': {0: 1},
  'probability': {0: -9.276970454273945}},
 '(one': {'class': [0],
  'frequency': {0: 1},
  'probability': {0: -9.276970454273945}},
 'minute': {'class': [0, 1],
  'frequency': {0: 3, 1: 1},
  'probability': {0: -8.583823273714, 1: -9.254405103138284}},
 'a': {'class': [0, 1],
  'frequency': {0: 324, 1: 359},
  'probability': {0: -4.186292452504153, 1: -4.061448252248073}},
 'killer,

In [ ]:
voc

In [27]:
voc['and']

{'class': [1, 0],
 'frequency': {0: 351, 1: 526},
 'probability': {0: -4.106486459235794, 1: -3.6803517351568664}}

In [142]:
import random
from math import *


class Data:

    def __init__(self, *files, quiet=True):
        """

        :param files:
        """
        self.files = files
        self.quiet = quiet
        self.total_data, \
        self.training_data, \
        self.test_data, \
        self.voc_classes, \
        self.doc_classes, \
        self.vocabulary = self.data_cutter()
        self.log_priors = self.log_prior()
        self.log_likelihood()

    def data_cutter(self):

        # code to cut data
        files = self.files
        total_data = []
        training_data = []
        test_data = []
        voc_classes = {}
        doc_classes = {}
        vocabulary = {}

        word_count = 0

        for f in files:
            data = open(f, "r")

            c = 0
            w = 0

            for line in data:
                t_line = line \
                    .rstrip('\n') \
                    .split('\t')

                label = int(t_line[1])
                sentence = t_line[0]

                if label not in doc_classes:
                    doc_classes[label] = 1
                else:
                    doc_classes[label] += 1

                # split words in each sentence.
                # TODO: tokenize and remove filler words
                words = sentence.split()

                doc = (label, words)

                total_data.append(doc)

                w += len(words)
            # TODO: create shuffle trigger
            random.shuffle(total_data)

            # TODO: create portioning variable
            training_data = total_data[0:int(0.8 * len(total_data))]
            test_data = total_data[int(0.8 * len(total_data)):]

            c += 1
            if not self.quiet:
                print('{} finished on line {} with {} words'
                      '\n==========================\n'.format(f, c, w))
            data.close()

            # exit file loop

        for doc in training_data:

            label = doc[0]
            words = doc[1]

            for word in words:
                if word not in vocabulary:
                    # initialising word in vocabulary
                    vocabulary[word] = {"frequency": {l: 0 for l in doc_classes}, "class": [label]}
                    vocabulary[word]["frequency"][label] += 1
                else:
                    if label in vocabulary[word]["class"]:
                        vocabulary[word]["frequency"][label] += 1

                    else:
                        vocabulary[word]["class"].append(label)
                        vocabulary[word]["frequency"][label] = 1

                if label not in voc_classes:
                    voc_classes[label] = 1
                else:
                    voc_classes[label] += 1

                word_count += 1

        #
        #     c += 1
        #
        # print('{} finished on line {} with {} words'
        #       '\n==========================\n'.format(f, c, w))
        if not self.quiet:
            print(
                "=====Summary=====\n"
                "Unique words:\t -> {}\n"
                "Total words:\t -> {}\n"
                "Total data:\t -> {}\n"
                "Train data:\t -> {}\n"
                "Test data:\t -> {}\n"
                    .format(len(vocabulary),
                            word_count,
                            len(total_data),
                            len(training_data),
                            len(test_data)))

        return total_data, training_data, test_data, voc_classes, doc_classes, vocabulary

    # def vocabulary(self):
    #     return self.vocabulary
    #
    # def word(self, word):
    #     return self.vocabulary[word]

    def log_likelihood(self):

        for word in self.vocabulary:
            self.vocabulary[word]["probability"] = {}
            for cls, wfr in self.vocabulary[word]["frequency"].items():
                cf = self.voc_classes[cls]

                p = log10((wfr + 1) / (cf + len(self.vocabulary)))

                self.vocabulary[word]["probability"][cls] = p
        if not self.quiet:
            print("Log probabilities calculated")

        return self

    def log_prior(self):
        log_priors = {}
        n_doc = len(self.training_data)
        for cls, n_c in self.doc_classes.items():
            pc = log10(n_c / n_doc)
            log_priors[cls] = pc

        return log_priors

    def nb_classifier(self, doc):
        sum_c = {}
        for cls, lp in self.log_priors.items():
            sum_c[cls] = lp
            for word in doc:
                if word in self.vocabulary:
                    sum_c[cls] += self.vocabulary[word]['probability'][cls]
        return max(sum_c.items())

    def test(self):
        score = 0
        word_count = 0
        for doc in self.test_data:
            words = doc[1]
            label = doc[0]

            result: tuple = self.nb_classifier(words)
            if result[0] == label:
                score += 1
            word_count += 1
        return (score / word_count) * 100


In [143]:
v2 = Data(amazon, imdb, yelp)
res = v2.test()
print(res)

50.5


In [ ]:
import time

runs = 100000
vals = []
start = time.time()
for i in range(runs):
    v2 = Data(amazon, imdb, yelp)
    res = v2.test()
    vals.append(res)
finish = time.time()-start
print("accuracy after {} runs:{} \ntime: {}"
      .format(runs, max(vals), finish))

In [122]:
v2.vocabulary['and']

{'class': [0, 1],
 'frequency': {0: 349, 1: 540},
 'probability': {0: -1.7787712283360455, 1: -1.5882768859239864}}

In [55]:
max(cls.items())

(1, 14027)

In [53]:
cls

{0: 14504, 1: 14027}

In [102]:
d = {'e':2}
r = {'a':1 , 'b':2, 'c':3, 'd':4}
p = 'a'
d = {l:0 for l in r}
d[p] = 1
#d['a'] = 0
d

{'a': 1, 'b': 0, 'c': 0, 'd': 0}

In [90]:
d['a']

0

In [116]:
voc = v2.vocabulary
for word, data in voc.items():
    print('{} -> {} \n'.format(word, data))

This -> {'frequency': {0: 53, 1: 76}, 'class': [1, 0], 'probability': {0: -2.5966174558843997, 1: -2.4382031886892928}} 

is -> {'frequency': {0: 255, 1: 330}, 'class': [1, 0], 'probability': {0: -1.920771250395519, 1: -1.804865920086056}} 

a -> {'frequency': {0: 316, 1: 363}, 'class': [1, 0], 'probability': {0: -1.827951953489617, 1: -1.7635925302127187}} 

witty -> {'frequency': {0: 0, 1: 1}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -4.023663918197793}} 

and -> {'frequency': {0: 364, 1: 536}, 'class': [1, 0], 'probability': {0: -1.7667183512508937, 1: -1.5947196281622191}} 

delightful -> {'frequency': {0: 0, 1: 2}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -3.8475726591421124}} 

adaptation -> {'frequency': {0: 0, 1: 2}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -3.8475726591421124}} 

of -> {'frequency': {0: 257, 1: 253}, 'class': [1, 0], 'probability': {0: -1.9173915097441383, 1: -1.9198601972418365}} 

the -> {'frequency': {0: 581, 1: 


Drago, -> {'frequency': {0: 0, 1: 1}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -4.023663918197793}} 

Stephen -> {'frequency': {0: 0, 1: 1}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -4.023663918197793}} 

McHattie -> {'frequency': {0: 0, 1: 1}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -4.023663918197793}} 

Lance -> {'frequency': {0: 0, 1: 1}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -4.023663918197793}} 

Hendrikson -> {'frequency': {0: 0, 1: 1}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -4.023663918197793}} 

together; -> {'frequency': {0: 0, 1: 1}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -4.023663918197793}} 

talk -> {'frequency': {0: 6, 1: 2}, 'class': [1, 0], 'probability': {0: -3.4839131756931114, 1: -3.8475726591421124}} 

raging -> {'frequency': {0: 0, 1: 1}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -4.023663918197793}} 

cheekbones!). -> {'frequency': {0: 0, 1: 1}, 'class

honest. -> {'frequency': {0: 1, 1: 1}, 'class': [1, 0], 'probability': {0: -4.0279812200433875, 1: -4.023663918197793}} 

That's -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

describe -> {'frequency': {0: 3, 1: 0}, 'class': [0], 'probability': {0: -3.7269512243794063, 1: -4.324693913861775}} 

painfully -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

dreary -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

time-waster -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

Razr -> {'frequency': {0: 0, 1: 2}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -3.8475726591421124}} 

buy. -> {'frequency': {0: 0, 1: 1}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -4.023663918197793}} 

roth's -> {'frequency': {0: 0, 1: 1}, 'class': 


style. -> {'frequency': {0: 0, 1: 1}, 'class': [1], 'probability': {0: -4.329011215707369, 1: -4.023663918197793}} 

vinegrette -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

baby -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

greens -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

hearts -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

Palm. -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

Filmiing -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

expansive. -> {'frequency': {0: 1, 1: 0}, 'class': [0], 'probability': {0: -4.0279812200433875, 1: -4.324693913861775}} 

Bland... -> {'frequency': {0: 1, 1: 0}, 'class': 